# EDA de BRENT


In [98]:
import pandas as pd
from azure.data.tables import TableClient
import os
from dotenv import load_dotenv
import plotly.express as px

In [99]:
load_dotenv()

True

In [100]:
connection_string = os.getenv("AZ_CONNECTION_STRING")

In [101]:
## Definiendo funciones:


# Crear conexión de una tabla específica dentro del servicio de Azure Table Storage
def set_table_service(connection_string, table):
    """Crear servicio de conexión a Azure Table Storage"""
    return TableClient.from_connection_string(
        conn_str=connection_string, table_name=table
    )


# Obtener datos de Table Storage
def get_data_from_table_storage_table(table_service, filter_query):
    """Recuperar datos de Table Storage"""
    for record in table_service.query_entities(filter_query):
        yield record


# Crear DataFrame con los datos de la tabla consultada
def get_dataframe_from_table_storage_table(table_service, filter_query):
    """Crear un DataFrame con la data del Table Storage"""
    return pd.DataFrame(get_data_from_table_storage_table(table_service, filter_query))

In [102]:
table_name_brent = "BRENTCurated"
filterQuery = "PartitionKey ne 'random'"

In [103]:
table_name_brent = set_table_service(connection_string, table_name_brent)
df_brent = get_dataframe_from_table_storage_table(table_name_brent, filterQuery)
print(f"Dimensión de la data: {df_brent.shape}")
print(f"Tipos de datos:\n{df_brent.dtypes}")
df_brent.head()

Dimensión de la data: (532, 9)
Tipos de datos:
PartitionKey    object
RowKey          object
Apertura        object
Fecha           object
Máximo          object
Mínimo          object
Vol             object
var             object
Último          object
dtype: object


,PartitionKey,RowKey,Apertura,Fecha,Máximo,Mínimo,Vol,var,Último
0,DefaultPartitionKey,000b65fc-8b3b-4e0b-9ba3-5f1842b1ecb0,"84,87",13.01.2022,"85,10","83,79","265,20K","-0,24%","84,47"
1,DefaultPartitionKey,001c0a9d-9441-4aff-94db-6ba29f786e96,"79,49",19.12.2022,"81,19","79,02","164,92K","0,82%","80,14"
2,DefaultPartitionKey,00604e4e-31f7-4de6-9a39-15ff4466953f,"99,79",31.08.2022,"100,46","95,44","10,07K","-1,38%","96,49"
3,DefaultPartitionKey,00666023-1642-4f3f-86c1-7f4a33c75a7e,"82,88",29.12.2022,"83,14","81,28","10,99K","-1,20%","82,26"
4,DefaultPartitionKey,007881a7-7471-40a4-aa2b-02720a06d866,"75,47",23.12.2021,"77,00","74,78","161,24K","2,07%","76,85"


## Quitamos `PartitionKey` y `RowKey`

Esto es metadata de Azure Storage así que no nos aporta nada


In [104]:
df_brent = df_brent.drop(["PartitionKey", "RowKey"], axis=1)

## Conversión de strings y casting de valores


In [105]:
df_brent["Máximo"] = df_brent["Máximo"].str.replace(",", ".")
df_brent["Mínimo"] = df_brent["Mínimo"].str.replace(",", ".")
df_brent["Último"] = df_brent["Último"].str.replace(",", ".")
df_brent["Máximo"] = df_brent["Máximo"].astype("float")
df_brent["Mínimo"] = df_brent["Mínimo"].astype("float")
df_brent["Último"] = df_brent["Último"].astype("float")

In [106]:
df_brent_sin_duplicados = df_brent.drop_duplicates(subset=["Fecha"])
brent_sin_duplicados_rows = df_brent_sin_duplicados.shape[0]
print(f"Número de registros después de quitar duplicados: {brent_sin_duplicados_rows}")
print(
    f"Tiene duplicados?: {'Sí' if brent_sin_duplicados_rows != df_brent.shape[0] else 'No'}"
)

Número de registros después de quitar duplicados: 532
Tiene duplicados?: No


Se concluye que BRENT no tiene duplicados


## Conversión de fecha

Hacemos que la fecha tenga el tipo de dato correcto (datetime64[ns])


In [107]:
df_brent_sin_duplicados_fecha = df_brent_sin_duplicados.copy()

df_brent_sin_duplicados_fecha["Fecha"] = pd.to_datetime(
    df_brent_sin_duplicados_fecha["Fecha"], errors="raise", dayfirst=True
)
df_brent_sin_duplicados_fecha["Fecha"] = df_brent_sin_duplicados_fecha["Fecha"]
print(f"Tipos de datos:\n{df_brent_sin_duplicados_fecha.dtypes}")
df_brent_sin_duplicados_fecha = df_brent_sin_duplicados_fecha.sort_values(by="Fecha")
df_brent_sin_duplicados_fecha.head(20)

Tipos de datos:
Apertura            object
Fecha       datetime64[ns]
Máximo             float64
Mínimo             float64
Vol                 object
var                 object
Último             float64
dtype: object


,Apertura,Fecha,Máximo,Mínimo,Vol,var,Último
201,"78,49",2021-10-01,79.41,77.55,"282,37K","0,97%",79.28
161,"79,49",2021-10-04,82.00,78.75,"362,37K","2,50%",81.26
374,"81,27",2021-10-05,83.13,81.19,"340,27K","1,60%",82.56
235,"82,57",2021-10-06,83.47,80.60,"401,36K","-1,79%",81.08
359,"80,98",2021-10-07,82.50,79.08,"393,10K","1,07%",81.95
275,"82,38",2021-10-08,83.43,81.96,"360,97K","0,54%",82.39
10,"82,65",2021-10-11,84.60,82.54,"330,44K","1,53%",83.65
483,"83,49",2021-10-12,84.23,82.72,"331,41K","-0,27%",83.42
424,"83,10",2021-10-13,83.69,82.20,"333,68K","-0,29%",83.18
257,"83,45",2021-10-14,84.50,83.18,"316,64K","0,99%",84.00


## Agregamos las fechas que faltan

Debido a que el BRENT no se cotiza en fines de semana, hay fechas que no están en el dataset. Se agregan con el valor del día anterior haciendo un backward fill.


In [108]:
df_brent_sin_duplicados_fecha = df_brent_sin_duplicados_fecha.set_index("Fecha")
df_brent_sin_duplicados_fecha = df_brent_sin_duplicados_fecha.asfreq("D")
df_brent_sin_duplicados_fecha = df_brent_sin_duplicados_fecha.ffill()
df_brent_sin_duplicados_fecha = df_brent_sin_duplicados_fecha.reset_index()
df_brent_sin_duplicados_fecha.head(20)

,Fecha,Apertura,Máximo,Mínimo,Vol,var,Último
0,2021-10-01,"78,49",79.41,77.55,"282,37K","0,97%",79.28
1,2021-10-02,"78,49",79.41,77.55,"282,37K","0,97%",79.28
2,2021-10-03,"78,49",79.41,77.55,"282,37K","0,97%",79.28
3,2021-10-04,"79,49",82.00,78.75,"362,37K","2,50%",81.26
4,2021-10-05,"81,27",83.13,81.19,"340,27K","1,60%",82.56
5,2021-10-06,"82,57",83.47,80.60,"401,36K","-1,79%",81.08
6,2021-10-07,"80,98",82.50,79.08,"393,10K","1,07%",81.95
7,2021-10-08,"82,38",83.43,81.96,"360,97K","0,54%",82.39
8,2021-10-09,"82,38",83.43,81.96,"360,97K","0,54%",82.39
9,2021-10-10,"82,38",83.43,81.96,"360,97K","0,54%",82.39


## Cálculo del valor promedio usando la columna "Mínimo" y "Máximo"


In [109]:
df = df_brent_sin_duplicados_fecha.copy()
df["Valor Promedio"] = df[["Mínimo", "Máximo"]].mean(axis=1)
df.head()

,Fecha,Apertura,Máximo,Mínimo,Vol,var,Último,Valor Promedio
0,2021-10-01,"78,49",79.41,77.55,"282,37K","0,97%",79.28,78.480
1,2021-10-02,"78,49",79.41,77.55,"282,37K","0,97%",79.28,78.480
2,2021-10-03,"78,49",79.41,77.55,"282,37K","0,97%",79.28,78.480
3,2021-10-04,"79,49",82.00,78.75,"362,37K","2,50%",81.26,80.375
4,2021-10-05,"81,27",83.13,81.19,"340,27K","1,60%",82.56,82.160


## Ordenamiento de los datos por fecha


In [110]:
df.sort_values(by="Fecha", inplace=True)
df.head()

,Fecha,Apertura,Máximo,Mínimo,Vol,var,Último,Valor Promedio
0,2021-10-01,"78,49",79.41,77.55,"282,37K","0,97%",79.28,78.480
1,2021-10-02,"78,49",79.41,77.55,"282,37K","0,97%",79.28,78.480
2,2021-10-03,"78,49",79.41,77.55,"282,37K","0,97%",79.28,78.480
3,2021-10-04,"79,49",82.00,78.75,"362,37K","2,50%",81.26,80.375
4,2021-10-05,"81,27",83.13,81.19,"340,27K","1,60%",82.56,82.160


In [111]:
df.tail()

,Fecha,Apertura,Máximo,Mínimo,Vol,var,Último,Valor Promedio
745,2023-10-16,"90,98",91.39,89.50,"283,25K","-1,36%",89.65,90.445
746,2023-10-17,"89,91",91.00,88.88,"252,83K","0,28%",89.90,89.940
747,2023-10-18,"91,25",93.00,90.60,"348,19K","1,78%",91.50,91.800
748,2023-10-19,"91,35",93.48,89.57,"309,00K","0,96%",92.38,91.525
749,2023-10-20,"93,23",93.79,91.66,"272,72K","-0,24%",92.16,92.725


## Gráfico de series de tiempo para valores numéricos


In [112]:
fig = px.line(df, x="Fecha", y=["Mínimo", "Máximo"])
fig.show()

Se puede ver que los precios del petróleo tuvieron un pico máximo a inicios del año 2022, y que desde ahí ha tenido una tendencia bajista.


## Quitamos columnas innecesarias


In [113]:
final_df = df.drop(["Mínimo", "Máximo", "Vol", "var", "Último", "Apertura"], axis=1)

In [114]:
final_df.head(20)

,Fecha,Valor Promedio
0,2021-10-01,78.480
1,2021-10-02,78.480
2,2021-10-03,78.480
3,2021-10-04,80.375
4,2021-10-05,82.160
5,2021-10-06,82.035
6,2021-10-07,80.790
7,2021-10-08,82.695
8,2021-10-09,82.695
9,2021-10-10,82.695


In [115]:
final_df.describe()

,Fecha,Valor Promedio
count,750,750.000000
mean,2022-10-10 12:00:00,90.011533
min,2021-10-01 00:00:00,68.210000
25%,2022-04-06 06:00:00,81.276250
50%,2022-10-10 12:00:00,86.202500
75%,2023-04-15 18:00:00,96.143750
max,2023-10-20 00:00:00,129.085000
std,NaN,12.508528


A partir del análisis exploratorio se puede ver que el valor promedio de BRET es de 90.04 USD. El precio mínimo es de 68.21 USD y el precio máximo llegó a 96.09 USD.


In [116]:
print(f"Tipos del dataframe final:\n{final_df.dtypes}")

Tipos del dataframe final:
Fecha             datetime64[ns]
Valor Promedio           float64
dtype: object


In [117]:
final_df.rename(columns={"Valor Promedio": "brent_value"}, inplace=True)

In [118]:
final_df.to_csv("processed_tables/brent.csv", index=False)